# Mesh to Mesh Registration Example

ITK natively supports image-to-image registration, which is a common operation for medical images with symmetry. Another common method of storing 3D volumetric data is to represent volume surfaces with meshes. In this example we seek to register two meshes using various metrics and optimization techniques built on top of ITK.

Registration classes are defined in the HASI `registration` submodule and built on top of the ITK Python wrapping. The `MeanSquaresRegistrar` and `DiffeoRegistrar` classes apply registration techniques to images derived from mesh inputs. Mesh registration is carried out with each class in this notebook on sample bone femur mesh data in the `examples/Data` folder. 

This notebook requires the following modules, which can be either acquired via `pip` or built alongside the ITK `master` branch:
- [ITK](https://github.com/InsightSoftwareConsortium/ITK/)
- [ITKBoneEnhancement](https://github.com/InsightSoftwareConsortium/ITKBoneEnhancement)
- [ITKMeshToPolyData](https://github.com/InsightSoftwareConsortium/ITKMeshToPolyData)

In [1]:
# Update sys.path to reference src/ modules
import os
import sys
import copy
import importlib
from urllib.request import urlretrieve

import itk
from itkwidgets import view, checkerboard, compare
from ipywidgets import FloatProgress, Label, HBox, VBox, FloatText, ColorPicker, Button
PATTERN_COUNT = 10

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
os.makedirs('Data', exist_ok=True)

In [3]:
FIXED_MESH_FILE = 'Data/901-L-mesh.vtk'
MOVING_MESH_FILE = 'Data/901-R-mesh.vtk'

MEANSQUARES_OUTPUT_FILE = 'Data/901-meansquares-registered.vtk'
DIFFEO_OUTPUT_FILE = 'Data/901-diffeo-registered.vtk'

In [4]:
# Download meshes
if not os.path.exists(FIXED_MESH_FILE):
    url = 'https://data.kitware.com/api/v1/file/5f9daaae50a41e3d1924dae1/download'
    urlretrieve(url, FIXED_MESH_FILE)
if not os.path.exists(MOVING_MESH_FILE):
    url = 'https://data.kitware.com/api/v1/file/5f9daaba50a41e3d1924dae9/download'
    urlretrieve(url, MOVING_MESH_FILE)

In [5]:
fixed_mesh = itk.meshread(FIXED_MESH_FILE, itk.F)
moving_mesh = itk.meshread(MOVING_MESH_FILE, itk.F)

### Compare images with ITKWidgets

We can use `view`, `compare`, and `checkerboard` to inspect mesh and image data.

In [6]:
view(geometries=[fixed_mesh,moving_mesh])

Template itk::PyVectorContainer<signedshort>
 already defined as <class 'itk.itkPyVectorContainerPython.itkPyVectorContainerSS'>
 is redefined as {cl}
Template itk::PyVectorContainer<unsignedchar>
 already defined as <class 'itk.itkPyVectorContainerPython.itkPyVectorContainerUC'>
 is redefined as {cl}
Template itk::PyVectorContainer<float>
 already defined as <class 'itk.itkPyVectorContainerPython.itkPyVectorContainerF'>
 is redefined as {cl}
Template itk::PyVectorContainer<double>
 already defined as <class 'itk.itkPyVectorContainerPython.itkPyVectorContainerD'>
 is redefined as {cl}
Template itk::PyVectorContainer<unsignedint>
 already defined as <class 'itk.itkPyVectorContainerPython.itkPyVectorContainerUI'>
 is redefined as {cl}
Template itk::PyVectorContainer<itk::Point<float,2>>
 already defined as <class 'itk.itkPyVectorContainerPython.itkPyVectorContainerPF2'>
 is redefined as {cl}
Template itk::PyVectorContainer<itk::Point<float,3>>
 already defined as <class 'itk.itkPyVectorC

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

# Mesh-To-Image Conversion in the Base Class
Mesh registration classes inheriting from `MeshToMeshRegistrar` implement unique registration algorithms. The base class includes common definitions for 3D mesh-to-mesh registration, abstract methods, and a mesh-to-image conversion method.

The `mesh_to_image` function takes in a 3D mesh and converts it into an ITK 3D image object. The spacing, origin, and size of the 3D image may be calculated from the minimum bounding box of the mesh or set from a reference image.

In [7]:
from src.registration.meshtomeshregistrar import MeshToMeshRegistrar
registrar = MeshToMeshRegistrar()

In [8]:
fixed_img = registrar.mesh_to_image(fixed_mesh)
moving_img = registrar.mesh_to_image(moving_mesh, fixed_img)

Comparing the meshes generated from the given meshes, we see that the two bone images are generally similar but do not exactly align. Registration will translate one mesh so that the two bones better coincide.

In [9]:
checkerboard(fixed_img, moving_img, pattern=PATTERN_COUNT)

## Run Mean Squares Image Registration

The `MeanSquaresRegistrar` class converts meshes to images and runs [Broyden–Fletcher–Goldfarb–Shanno Optimization](https://itk.org/Doxygen/html/classitk_1_1LBFGSBOptimizerv4.html) on a `BSplineTransform` to iteratively reduce the mean square error. The resulting transform is then applied to resample the target mesh into the template mesh domain.

Progress is shown with an itkwidgets display via hooks into the ITK event-observer system. The resultant mesh is returned as an object in the Python environment and may be optionally written out to a file. Iteration updates may also be printed to the output window with the optional `verbose` flag.

In [10]:
from src.registration.meansquaresregistrar import MeanSquaresRegistrar

In [11]:
# Must instantiate a registration object to initialize optimizers
registrar = MeanSquaresRegistrar()

In [12]:
progress = FloatProgress(
        min=0.0,
        max=150.0,
        step=1
    )
box = HBox([
    Label('Register images'),
    progress
])
box

In [13]:
def update_progress():
    progress.value = registrar.optimizer.GetCurrentIteration()
registrar.optimizer.AddObserver(itk.IterationEvent(), update_progress)

0

In [14]:
meansquares_mesh_result = registrar.register(template_mesh=fixed_mesh,
                                             target_mesh=moving_mesh,
                                             filepath=MEANSQUARES_OUTPUT_FILE,
                                             num_iterations=200,
                                             verbose=True)

0 0.03902562455535499 0.0
0 0.02843709659649114 0.0
0 0.02843709659649114 0.0
1 0.021534717551505798 0.0009582783977029718
1 0.021534717551505798 0.0009582783977029718
2 0.01790224911093285 0.00043193967453070587
2 0.01790224911093285 0.00043193967453070587
3 0.01568735406387658 0.00024653588676924737
3 0.01568735406387658 0.00024653588676924737
4 0.012572812905498177 0.0001938980902412675
4 0.012572812905498177 0.0001938980902412675
5 0.010982249945285367 0.00015319597818792382
5 0.010982249945285367 0.00015319597818792382
6 0.010363658471730493 0.0001203197280705292
6 0.010363658471730493 0.0001203197280705292
7 0.00917110184613044 0.00012346212100002512
7 0.00917110184613044 0.00012346212100002512
8 0.008587382362441122 0.00010581326627687776
8 0.008587382362441122 0.00010581326627687776
9 0.007903788433334932 0.00023582429403319085
9 0.007903788433334932 0.00023582429403319085
10 0.007710042007798568 7.71329623659586e-05
10 0.007710042007798568 7.71329623659586e-05
11 0.00748263845

136 0.0018561265960633036 5.262116183299268e-06
136 0.0018561265960633036 5.262116183299268e-06
137 0.0018541785149490267 4.266601313887413e-06
137 0.0018541785149490267 4.266601313887413e-06
138 0.0018488759253997083 5.7777829698971234e-06
138 0.0018488759253997083 5.7777829698971234e-06
139 0.0018488214728622956 8.851509846565252e-06
139 0.0018488214728622956 8.851509846565252e-06
140 0.0018455469378301228 2.599417286662253e-05
140 0.0018455469378301228 2.599417286662253e-05
141 0.001842810237598681 6.118751047386717e-06
141 0.001842810237598681 6.118751047386717e-06
142 0.0018381434535997365 4.980478460443563e-06
142 0.0018381434535997365 4.980478460443563e-06
143 0.0018345788639350415 6.298596212917168e-06
143 0.0018345788639350415 6.298596212917168e-06
144 0.0018341905117658917 9.925644451932482e-06
144 0.0018341905117658917 9.925644451932482e-06
145 0.0018275382956829034 2.106023039839816e-05
145 0.0018275382956829034 2.106023039839816e-05
146 0.001826388215684216 1.0988414675348

Comparison of the resulting mesh with the target shows successful registration.

In [15]:
view(geometries=[meansquares_mesh_result,moving_mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

Comparison of the resulting mesh with the output file shows congruency.

In [16]:
file_mesh_result = itk.meshread(MEANSQUARES_OUTPUT_FILE, itk.F)

In [17]:
view(geometries=[file_mesh_result,meansquares_mesh_result])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

## Diffeomorphic Registration

The `DiffeoRegistrar` class converts meshes to images and performs registration using the [Diffeomorphic Demons registration algorithm](https://itk.org/Doxygen/html/classitk_1_1DiffeomorphicDemonsRegistrationFilter.html). The resultant deformation field is then applied to resample the target mesh into the template mesh domain.

Custom observers may print out iteration data accessed via the `registrar.filter` object.

In [18]:
from src.registration.diffeoregistrar import DiffeoRegistrar

In [19]:
registrar = DiffeoRegistrar()

In [20]:
diffeoProgress = FloatProgress(
        min=0.0,
        max=200.0,
        step=1
    )
diffeoBox = HBox([
    Label('Register images'),
    diffeoProgress
])
diffeoBox

In [21]:
def update_diff_progress():
    diffeoProgress.value = registrar.filter.GetElapsedIterations()

registrar.filter.AddObserver(itk.IterationEvent(),update_diff_progress)

diffeo_mesh_result = registrar.register(fixed_mesh,moving_mesh,filepath=DIFFEO_OUTPUT_FILE,verbose=True)

1.7976931348623157e+308
0.03902562456210709
0.03620058111051528
0.033857400476957365
0.0316529992799868
0.029671274883454234
0.027863336769245965
0.026263721371434446
0.024784571211718676
0.023439112890373832
0.022179969985074027
0.021160217575979016
0.02022989923756486
0.019367058349974004
0.018567845127124925
0.017848969406583762
0.017187661496342346
0.01657368277679982
0.01599817637581068
0.01545783158564288
0.014948025199071284
0.014461622847641747
0.014000352105777528
0.013558082766414237
0.01311751745327867
0.012612455200814072
0.012162631078637215
0.011740243666151801
0.011309865240838799
0.010899203476967512
0.010521650813570736
0.010196561107985157
0.009891130130343637
0.009600816910980126
0.009315760503390664
0.009039516487389243
0.008767196010894807
0.008506293948833205
0.008248633969299386
0.00799543225258579
0.0077453645420522965
0.007497301744479739
0.007257772289115741
0.007027261536318483
0.0068053907333040725
0.006592009351901685
0.006385407763355722
0.0061888179732270

Compare translated image to target image.

In [22]:
view(geometries=[diffeo_mesh_result,moving_mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

Comparison of the resulting mesh with the output file shows congruency.

In [23]:
file_mesh_result = itk.meshread(DIFFEO_OUTPUT_FILE, itk.F)
view(geometries=[file_mesh_result,diffeo_mesh_result])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

### Clean up example files

In [24]:
os.remove(MEANSQUARES_OUTPUT_FILE)
os.remove(DIFFEO_OUTPUT_FILE)